<a href="https://colab.research.google.com/github/pdenailly/Atelier_bikes/blob/main/ACTIVITE1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ANALYSE DE BASES DE DONNEES A L'AIDE DE STATISTIQUES SIMPLES**

Application : Etude des volumes de cyclistes dans la ville de Paris en amont, pendant et après les JO 2024.


Bienvenu dans cet atelier ! Nous allons y explorer quelques manières simples d'étudier un jeux de données avant toute utilisation pour des travaux de modélisation.

Pour ce cas pratique nous nous intéresserons à des séries temporelles de traffic de vélo, comptés à chaque heure entre juin et septembre 2024 en plusieurs points de la ville de Paris. Ces données ont été extraites d'une [base ouverte mise à disposition par la ville de Paris](https://parisdata.opendatasoft.com/explore/dataset/comptage-velo-donnees-compteurs/api/?disjunctive.id_compteur&disjunctive.nom_compteur&disjunctive.id&disjunctive.name&q.timerange.date=date:%5B2024-05-31T22:00:00Z+TO+2024-09-13T21:59:59Z%5D&basemap=jawg.dark&location=13,48.85652,2.35024&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQVZHIiwieUF4aXMiOiJzdW1fY291bnRzIiwic2NpZW50aWZpY0Rpc3BsYXkiOnRydWUsImNvbG9yIjoiI0ZBOEM0NCJ9XSwieEF4aXMiOiJkYXRlIiwibWF4cG9pbnRzIjoiIiwidGltZXNjYWxlIjoibW9udGgiLCJzb3J0IjoiIiwiY29uZmlnIjp7ImRhdGFzZXQiOiJjb21wdGFnZS12ZWxvLWRvbm5lZXMtY29tcHRldXJzIiwib3B0aW9ucyI6eyJkaXNqdW5jdGl2ZS5pZF9jb21wdGV1ciI6dHJ1ZSwiZGlzanVuY3RpdmUubm9tX2NvbXB0ZXVyIjp0cnVlLCJkaXNqdW5jdGl2ZS5pZCI6dHJ1ZSwiZGlzanVuY3RpdmUubmFtZSI6dHJ1ZSwicmVmaW5lLmRhdGUiOiIyMDI0In19fV0sImRpc3BsYXlMZWdlbmQiOnRydWUsImFsaWduTW9udGgiOnRydWUsInRpbWVzY2FsZSI6IiJ9). La période de Juillet 2024 a été marquée par la présence des Jeux Olympiques en région parisienne, l'un des mots d'ordres ayant été l'utilisation des mobilités douces, dont le vélo. A travers les différents ateliers proposés ici, nous allons nous intéresser à l'étude de ces données, et observer à quel point les jeux olympiques ont eu un impact sur le volume de cyclistes à Paris.

Assurez vous que le fichier de données *comptage-velo-donnees-compteurs.csv* soit présent dans le répertoire courant (sur colab, faites les glisser dans l'espace de stockage de la session avec le logo dossier à gauche). Ces fichiers sont disponibles sur le dépôt git de l'atelier : https://github.com/pdenailly/Atelier-Summer-School.


**Importation des librairies Python**

Nous allons travailler avec le language Python dans cet atelier. En amont de tout projet, il est toujours bon de rassembler les importations de librairies dans une section dédiée. Nous allons notamment travailler avec la librairie *pandas*, très adaptée à l'étude de bases de données sous python.

In [ ]:
import pandas as pd

**Importation des données de comptages de vélos**

L'importation des données se fait avec la librairie pandas, via la fonction read_csv(.).

In [ ]:
#Importation des données de comptage vélos depuis le fichier .csv
bike_data = pd.read_csv('comptage-velo-donnees-compteurs.csv', sep=";", decimal='.').iloc[: , 1:]


Maintenant que les données ont été importées sous python, il s'agit de les comprendre. Pour commencer nous devons nous faire une idée du contenu de la base, ses dimensions, ses types de variables, etc.

**Etudes préliminaires sur la base de données**

Commençons par chercher le nombre de lignes et de colonnes dans cette table afin d'en étudier les dimensions. Trouvez l'attrubut "shape" sur la dataframe pour cela. Avec pandas un attribut de dataframe 'att' est visualisable via la syntaxe 'dataframe.att'.

In [ ]:
bike_data.shape

(243730, 7)

Shape renvoie un objet *tuple* contenant le nombre de lignes et le nombre de colonnes. Qu'observez vous ici ?

Maintenant explorons de quelles variables (colonnes) nous disposons dans cette dataframe. Utilisez l'attribut "columns" pour cela.

In [ ]:
bike_data.columns

Index(['Nom du compteur', 'Identifiant du site de comptage',
       'Nom du site de comptage', 'Comptage horaire',
       'Date et heure de comptage', 'Date d'installation du site de comptage',
       'Coordonnées géographiques'],
      dtype='object')

Cette commande nous permet d'obtenir une liste de des noms de colonnes présentes dans la dataframe. Certaines comme 'Comptage horaire', 'Date et heure de comptage' et 'Nom du compteur' semblent particulièrement intéressantes comme elles pourront situer les comptages temporellement et spatialement.

Nous ne savons en revanche pas quels sont les types des colonnes (nombres entiers, continus, chaînes de caractère,etc.). Cette étape est importante, nous voulons effectivement connaitre la nature des colonnes afin de pouvoir les traiter comme il faut dans les analyses ultérieures. Vous trouverez dans la [documentation de pandas](https://pandas.pydata.org/docs/reference/frame.html) l'attribut nécessaires pour trouver le type de chaque colonne.

In [ ]:
bike_data.dtypes

,0
Nom du compteur,object
Identifiant du site de comptage,float64
Nom du site de comptage,object
Comptage horaire,int64
Date et heure de comptage,object
Date d'installation du site de comptage,object
Coordonnées géographiques,object


Le type object dans pandas est un type générique qui est utilisé pour stocker des données non numériques. En pratique, il est souvent utilisé pour représenter des chaines de caractère, ou bien des valeurs hétérogènes.

Nous observons notamment que la colonne 'Comptage horaire' est bien composée de comptages (int64). Les autres colonnes sont composées d'informations sous la forme de chaines de caractères, excepté la colonne 'Identifiant du site de comptage' qui est composée de valeurs numériques continues.

La fonction 'describe()' peut être appliquée sur une colonne du dataframe pour en décrire le contenu (nombre de valeurs, moyenne, min, max, etc.). Appliquez cette fonction à la colonne 'Comptage horaire' afin d'en étudier le contenu. Quelles sont les valeurs de moyenne ? De médiane ? Pourquoi sont elles différentes ?

In [ ]:
bike_data['Comptage horaire'].describe()

,Comptage horaire
count,243730.000000
mean,82.002765
std,113.475929
min,0.000000
25%,12.000000
50%,45.000000
75%,102.000000
max,1574.000000


Appliquez maintenant la fonction de description à la colonne 'Identifiant du site de comptage'. Qu'observez vous ? Est ce cohérent pour une colonne dont la donnée est censée donner un identifiant à chaque site de comptage ?



In [ ]:
bike_data['Identifiant du site de comptage'].describe()


,Identifiant du site de comptage
count,226264.0
unique,65.0
top,100057445.0
freq,5040.0


Modifiez la colonne 'Identifiant du site de comptage' afin d'en obtenir une variable catégorielle (chaque identifiant = une catégorie). Appliquez pour cela la fonction 'astype(...)' sur la colonne, et spécifiez que la transformation doit se faire vers des données catégorielles. Créez la colonne 'Identifiant du site de comptage cat' avec votre transformation. Ensuite appliquez la fonction de description à votre colonne modifiée 'Identifiant du site de comptage cat'. Qu'observez vous ? Combien y a t-il de sites de comptages ?

In [ ]:
bike_data['Identifiant du site de comptage'] = bike_data['Identifiant du site de comptage'].astype('category')
bike_data['Identifiant du site de comptage'].describe()


,Identifiant du site de comptage
count,226264.0
unique,65.0
top,100057445.0
freq,5040.0


Un site de comptage est une zone qui peut rassembler plusieurs capteurs de comptages répartis dans la ville de Paris. Les capteurs apparaissent dans la colonne 'Nom du compteur'. Combien y a t-il de capteurs en tout ? Vous pouvez passer par l'utilisation de la fonction 'nunique()'.

In [ ]:
bike_data['Nom du compteur'].nunique()


97

A ce stade, nous ne nous sommes pas intéressé à l'aspect temporel dans les données, pourtant central pour l'étude de comptages de vélos dans le temps.